In [1]:
# -----------------------------
# Imports
# -----------------------------
import os
import numpy as np
import pandas as pd
import gradio as gr
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# -----------------------------
# Config
# -----------------------------
CLASS_NAMES = ["Mpox", "Chickenpox", "Measles", "Cowpox", "Healthy", "Hand Foot and Mouth"]

MODEL_PATHS = {
    "DenseNet201": "/kaggle/input/mpoxdetectionmodels/tensorflow2/default/1/NewestModels/Bonolo_Mpox_DenseNet201_model.h5",
    "ResNet152V2": "/kaggle/input/mpoxdetectionmodels/tensorflow2/default/1/NewestModels/Bonolo_Mpox_ResNet152V2_model.h5",
    "InceptionV3": "/kaggle/input/mpoxdetectionmodels/tensorflow2/default/1/NewestModels/Bonolo_Mpox_InceptionNet_model.h5",
    "XceptionNet": "/kaggle/input/mpoxdetectionmodels/tensorflow2/default/1/NewestModels/Bonolo_Mpox_Xception_model.h5"
}

LAST_CONV_LAYERS = {
    "DenseNet201": None,
    "ResNet152V2": None,
    "InceptionV3": "mixed10",
    "XceptionNet": None
}

LOADED_MODELS = {}

# -----------------------------
# Load model
# -----------------------------
def get_model(model_name: str):
    if model_name not in LOADED_MODELS:
        LOADED_MODELS[model_name] = load_model(MODEL_PATHS[model_name], compile=False)
    return LOADED_MODELS[model_name]

# -----------------------------
# Preprocess function
# -----------------------------
def preprocess(img: np.ndarray, target_size=(224, 224)):
    img = image.array_to_img(img).resize(target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

# -----------------------------
# HSV augmentation
# -----------------------------
def hsv_color_space_augmentation(image, num_augments=5):
    augmented_images = []
    for i in range(num_augments):
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
        hue_shift = i % 180
        hsv[..., 0] = (hsv[..., 0] + hue_shift) % 180
        sat_mult = np.random.uniform(0.9, 1.1)
        val_mult = np.random.uniform(0.9, 1.1)
        hsv[..., 1] *= sat_mult
        hsv[..., 2] *= val_mult
        hsv = np.clip(hsv, 0, 255).astype(np.uint8)
        aug_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        augmented_images.append(aug_img)
    return augmented_images

# -----------------------------
# Prediction
# -----------------------------
def predict(model_name, img, hsv_augment=False, num_augments=5):
    model = get_model(model_name)
    if hsv_augment:
        imgs = hsv_color_space_augmentation(img, num_augments)
        preds_list = [model.predict(preprocess(im))[0] for im in imgs]
        preds = np.mean(preds_list, axis=0)
    else:
        preds = model.predict(preprocess(img))[0]

    idx = int(np.argmax(preds))
    pred_class = CLASS_NAMES[idx]

    prob_df = pd.DataFrame({"class": CLASS_NAMES, "probability": preds})\
        .sort_values("probability", ascending=False).reset_index(drop=True)
    return pred_class, prob_df

# -----------------------------
# Find last conv layer
# -----------------------------
def find_last_conv_layer(model):
    conv_types = (tf.keras.layers.Conv2D, tf.keras.layers.SeparableConv2D, tf.keras.layers.DepthwiseConv2D)
    for layer in reversed(model.layers):
        if isinstance(layer, conv_types):
            return layer.name
    for layer in reversed(model.layers):
        if 'conv' in layer.name.lower():
            return layer.name
    raise ValueError("No convolutional layer found.")

# -----------------------------
# Grad-CAM
# -----------------------------
def grad_cam(img, model, class_idx=None, layer_name=None, hsv_augment=False, num_augments=5):
    if layer_name is None:
        layer_name = find_last_conv_layer(model)

    if hsv_augment and num_augments > 1:
        aug_images = hsv_color_space_augmentation(img, num_augments)
        heatmaps = []
        for aug_img in aug_images:
            heatmaps.append(_single_grad_cam(aug_img, model, class_idx, layer_name))
        avg_heatmap = np.mean(np.array(heatmaps), axis=0)
    else:
        avg_heatmap = _single_grad_cam(img, model, class_idx, layer_name)

    avg_heatmap_color = cv2.applyColorMap(np.uint8(255*avg_heatmap), cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 0.6, avg_heatmap_color, 0.4, 0)
    return superimposed_img

def _single_grad_cam(img, model, class_idx, layer_name):
    img_array = preprocess(img)
    preds = model.predict(img_array)
    if class_idx is None:
        class_idx = int(np.argmax(preds[0]))
    conv_layer = model.get_layer(layer_name)
    inputs = model.inputs if isinstance(model.inputs, list) else [model.inputs]
    grad_model = tf.keras.models.Model(inputs, [conv_layer.output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_idx]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0].numpy()
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:,:,i] *= pooled_grads[i]
    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) if np.max(heatmap) != 0 else 1
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    return heatmap

# -----------------------------
# Combined predict + Grad-CAM
# -----------------------------
def predict_with_gradcam(model_name, img, hsv_augment=False, num_augments=5):
    try:
        model = get_model(model_name)
        pred_class, prob_df = predict(model_name, img, hsv_augment, num_augments)
        heatmap_img = grad_cam(img, model, layer_name=LAST_CONV_LAYERS.get(model_name),
                               hsv_augment=hsv_augment, num_augments=num_augments)
        return pred_class, prob_df, heatmap_img
    except Exception as e:
        blank_df = pd.DataFrame({"class": CLASS_NAMES, "probability": [0.0]*len(CLASS_NAMES)})
        blank_image = np.zeros((224,224,3), dtype=np.uint8)
        return f"Error: {e}", blank_df, blank_image

# -----------------------------
# Gradio UI
# -----------------------------
with gr.Blocks(title="Multi-Model Mpox Classifier") as demo:
    gr.Markdown("# 🦠 Skin Lesion Classifier\nChoose a model, upload an image, and see predictions with Grad-CAM.")

    with gr.Row():
        with gr.Column(scale=3):
            model_choice = gr.Dropdown(choices=list(MODEL_PATHS.keys()), value=list(MODEL_PATHS.keys())[0],
                                       label="Choose Model")
            img_input = gr.Image(type="numpy", label="Upload Image")
            hsv_toggle = gr.Checkbox(label="Enable HSV Augmentation", value=False)
            num_augments = gr.Slider(1, 50, value=5, step=1, label="Number of HSV Augments")
            predict_btn = gr.Button("Predict")

        with gr.Column(scale=2):
            pred_label = gr.Label(label="Predicted Class")
            prob_table = gr.Dataframe(headers=["class","probability"], row_count=(len(CLASS_NAMES),"fixed"))
            gradcam_output = gr.Image(type="numpy", label="Grad-CAM Heatmap")

    predict_btn.click(fn=predict_with_gradcam,
                      inputs=[model_choice, img_input, hsv_toggle, num_augments],
                      outputs=[pred_label, prob_table, gradcam_output])

if __name__ == "__main__":
    demo.launch()

2025-11-16 04:25:20.319896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763267120.581794      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763267120.656382      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://1e252e6b80f3ae9dbf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
# -----------------------------
# Imports
# -----------------------------
import os
import numpy as np
import pandas as pd
import gradio as gr
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from lime import lime_image
from skimage.segmentation import mark_boundaries

# -----------------------------
# Config
# -----------------------------
CLASS_NAMES = ["Mpox", "Chickenpox", "Measles", "Cowpox", "Healthy", "Hand Foot and Mouth"]

MODEL_PATHS = {
    "DenseNet201": "/kaggle/input/mpoxdetectionmodels/tensorflow2/default/1/NewestModels/Bonolo_Mpox_DenseNet201_model.h5",
    "ResNet152V2": "/kaggle/input/mpoxdetectionmodels/tensorflow2/default/1/NewestModels/Bonolo_Mpox_ResNet152V2_model.h5",
    "InceptionV3": "/kaggle/input/mpoxdetectionmodels/tensorflow2/default/1/NewestModels/Bonolo_Mpox_InceptionNet_model.h5",
    "XceptionNet": "/kaggle/input/mpoxdetectionmodels/tensorflow2/default/1/NewestModels/Bonolo_Mpox_Xception_model.h5"
}

LAST_CONV_LAYERS = {
    "DenseNet201": None,
    "ResNet152V2": None,
    "InceptionV3": "mixed10",
    "XceptionNet": None
}

LOADED_MODELS = {}

# -----------------------------
# Load model
# -----------------------------
def get_model(model_name: str):
    if model_name not in LOADED_MODELS:
        LOADED_MODELS[model_name] = load_model(MODEL_PATHS[model_name], compile=False)
    return LOADED_MODELS[model_name]

# -----------------------------
# Preprocess function
# -----------------------------
def preprocess(img: np.ndarray, target_size=(224, 224)):
    img = image.array_to_img(img).resize(target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

# -----------------------------
# HSV augmentation
# -----------------------------
def hsv_color_space_augmentation(image, num_augments=5):
    augmented_images = []
    for i in range(num_augments):
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
        hue_shift = i % 180
        hsv[..., 0] = (hsv[..., 0] + hue_shift) % 180
        sat_mult = np.random.uniform(0.9, 1.1)
        val_mult = np.random.uniform(0.9, 1.1)
        hsv[..., 1] *= sat_mult
        hsv[..., 2] *= val_mult
        hsv = np.clip(hsv, 0, 255).astype(np.uint8)
        aug_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        augmented_images.append(aug_img)
    return augmented_images

# -----------------------------
# Prediction
# -----------------------------
def predict(model_name, img, hsv_augment=False, num_augments=5):
    model = get_model(model_name)
    if hsv_augment:
        imgs = hsv_color_space_augmentation(img, num_augments)
        preds_list = [model.predict(preprocess(im))[0] for im in imgs]
        preds = np.mean(preds_list, axis=0)
    else:
        preds = model.predict(preprocess(img))[0]

    idx = int(np.argmax(preds))
    pred_class = CLASS_NAMES[idx]

    prob_df = pd.DataFrame({"class": CLASS_NAMES, "probability": preds})\
        .sort_values("probability", ascending=False).reset_index(drop=True)
    return pred_class, prob_df

# -----------------------------
# Find last conv layer
# -----------------------------
def find_last_conv_layer(model):
    conv_types = (tf.keras.layers.Conv2D, tf.keras.layers.SeparableConv2D, tf.keras.layers.DepthwiseConv2D)
    for layer in reversed(model.layers):
        if isinstance(layer, conv_types):
            return layer.name
    for layer in reversed(model.layers):
        if 'conv' in layer.name.lower():
            return layer.name
    raise ValueError("No convolutional layer found.")

# -----------------------------
# Grad-CAM
# -----------------------------
def grad_cam(img, model, class_idx=None, layer_name=None, hsv_augment=False, num_augments=5):
    if layer_name is None:
        layer_name = find_last_conv_layer(model)

    if hsv_augment and num_augments > 1:
        aug_images = hsv_color_space_augmentation(img, num_augments)
        heatmaps = []
        for aug_img in aug_images:
            heatmaps.append(_single_grad_cam(aug_img, model, class_idx, layer_name))
        avg_heatmap = np.mean(np.array(heatmaps), axis=0)
    else:
        avg_heatmap = _single_grad_cam(img, model, class_idx, layer_name)

    avg_heatmap_color = cv2.applyColorMap(np.uint8(255*avg_heatmap), cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 0.6, avg_heatmap_color, 0.4, 0)
    return superimposed_img

def _single_grad_cam(img, model, class_idx, layer_name):
    img_array = preprocess(img)
    preds = model.predict(img_array)
    if class_idx is None:
        class_idx = int(np.argmax(preds[0]))
    conv_layer = model.get_layer(layer_name)
    inputs = model.inputs if isinstance(model.inputs, list) else [model.inputs]
    grad_model = tf.keras.models.Model(inputs, [conv_layer.output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_idx]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0].numpy()
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:,:,i] *= pooled_grads[i]
    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) if np.max(heatmap) != 0 else 1
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    return heatmap

# -----------------------------
# LIME Explanation
# -----------------------------
def lime_explanation(img, model):
    explainer = lime_image.LimeImageExplainer()

    def predict_fn(images):
        images = np.array([im/255.0 for im in images])
        return model.predict(images)

    explanation = explainer.explain_instance(
        image.img_to_array(image.array_to_img(img).resize((224, 224))).astype('double'),
        predict_fn,
        top_labels=1,
        hide_color=0,
        num_samples=1000
    )

    temp, mask = explanation.get_image_and_mask(
        label=explanation.top_labels[0],
        positive_only=False,
        num_features=10,
        hide_rest=False
    )

    lime_image_result = mark_boundaries(temp / 255.0, mask)
    return (lime_image_result * 255).astype(np.uint8)

# -----------------------------
# Combined predict + Grad-CAM + LIME
# -----------------------------
def predict_with_explainability(model_name, img, hsv_augment=False, num_augments=5):
    try:
        model = get_model(model_name)
        pred_class, prob_df = predict(model_name, img, hsv_augment, num_augments)

        # Grad-CAM visualization
        gradcam_img = grad_cam(
            img, model, layer_name=LAST_CONV_LAYERS.get(model_name),
            hsv_augment=hsv_augment, num_augments=num_augments
        )

        # LIME explanation
        lime_img = lime_explanation(img, model)

        return pred_class, prob_df, gradcam_img, lime_img

    except Exception as e:
        blank_df = pd.DataFrame({"class": CLASS_NAMES, "probability": [0.0]*len(CLASS_NAMES)})
        blank_image = np.zeros((224,224,3), dtype=np.uint8)
        return f"Error: {e}", blank_df, blank_image, blank_image

# -----------------------------
# Gradio UI
# -----------------------------
with gr.Blocks(title="Multi-Model Mpox Classifier with Explainability") as demo:
    gr.Markdown("# 🦠 Skin Lesion Classifier\n### Choose a model, upload an image, and see predictions with Grad-CAM and LIME explanations.")

    with gr.Row():
        with gr.Column(scale=3):
            model_choice = gr.Dropdown(choices=list(MODEL_PATHS.keys()), value=list(MODEL_PATHS.keys())[0],
                                       label="Choose Model")
            img_input = gr.Image(type="numpy", label="Upload Image")
            hsv_toggle = gr.Checkbox(label="Enable HSV Augmentation", value=False)
            num_augments = gr.Slider(1, 50, value=5, step=1, label="Number of HSV Augments")
            predict_btn = gr.Button("Predict")

        with gr.Column(scale=2):
            pred_label = gr.Label(label="Predicted Class")
            prob_table = gr.Dataframe(headers=["class","probability"], row_count=(len(CLASS_NAMES),"fixed"))
            gradcam_output = gr.Image(type="numpy", label="Grad-CAM Heatmap")
            lime_output = gr.Image(type="numpy", label="LIME Explanation")

    predict_btn.click(
        fn=predict_with_explainability,
        inputs=[model_choice, img_input, hsv_toggle, num_augments],
        outputs=[pred_label, prob_table, gradcam_output, lime_output]
    )

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7861
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://6ff5955f9c2852beb5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


2025-11-16 04:27:20.141133: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 